## Inference

In [1]:
import pandas as pd
import numpy as np
import pickle
from datetime import timedelta
import calendar
import tensorflow as tf

In [2]:
SEASONAL_MODEL_PATH = 'weather_seasonal_model.keras'
SEASONAL_PREPROCESSOR_PATH = 'weather_seasonal_preprocessor.pkl'

In [3]:
def predict_weather_range(kecamatan_name: str, start_date_str: str, days_to_predict: int):

    # load model and preprocessor
    try:
        model = tf.keras.models.load_model(SEASONAL_MODEL_PATH)
        with open(SEASONAL_PREPROCESSOR_PATH, 'rb') as f:
            preprocessor = pickle.load(f)
    except FileNotFoundError as e:
        return f"Error: Could not load model or preprocessor. Make sure files exist. Details: {e}"

    # generate feature for range
    try:
        start_date = pd.to_datetime(start_date_str)
    except ValueError:
        return "Error: Invalid date format. Please use 'YYYY-MM-DD'."

    features_list = []
    date_range = [start_date + timedelta(days=i) for i in range(days_to_predict)]

    for date in date_range:
        features_list.append({
            'year': date.year,
            'day_sin': np.sin(2 * np.pi * date.dayofyear / 366),
            'day_cos': np.cos(2 * np.pi * date.dayofyear / 366),
            'month_sin': np.sin(2 * np.pi * date.month / 12),
            'month_cos': np.cos(2 * np.pi * date.month / 12),
            'kecamatan': kecamatan_name
        })
    
    input_df = pd.DataFrame(features_list)

    # preprocess and predict
    try:
        input_processed = preprocessor.transform(input_df)
    except Exception as e:
        return f"Error: Could not process input. It's possible the location '{kecamatan_name}' was not in the training data. Error: {e}"

    all_predicted_values = model.predict(input_processed)

    # format output
    final_results = []
    targets = ['precipprob', 'windspeed', 'temp', 'humidity']
    
    for i, date in enumerate(date_range):
        result = {
            'date': date.strftime('%Y-%m-%d'),
            'kecamatan': kecamatan_name,
        }
        predicted_values_for_day = all_predicted_values[i]
        
        for target, value in zip(targets, predicted_values_for_day):
            result[f"predicted_{target}"] = round(float(value), 2)
        
        final_results.append(result)
        
    return final_results

In [4]:
predict_range = predict_weather_range(
    kecamatan_name="berastagi",
    start_date_str="2025-06-06",
    days_to_predict=3
)

predict_range

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 282ms/step


[{'date': '2025-06-06',
  'kecamatan': 'berastagi',
  'predicted_precipprob': 63.47,
  'predicted_windspeed': 19.22,
  'predicted_temp': 28.2,
  'predicted_humidity': 87.74},
 {'date': '2025-06-07',
  'kecamatan': 'berastagi',
  'predicted_precipprob': 63.52,
  'predicted_windspeed': 19.24,
  'predicted_temp': 28.19,
  'predicted_humidity': 87.78},
 {'date': '2025-06-08',
  'kecamatan': 'berastagi',
  'predicted_precipprob': 63.56,
  'predicted_windspeed': 19.25,
  'predicted_temp': 28.17,
  'predicted_humidity': 87.83}]

In [5]:
def predict_monthly_average(kecamatan_name: str, month: int, year: int):

    # load model and preprocessor
    try:
        model = tf.keras.models.load_model(SEASONAL_MODEL_PATH)
        with open(SEASONAL_PREPROCESSOR_PATH, 'rb') as f:
            preprocessor = pickle.load(f)
    except FileNotFoundError as e:
        return f"Error: Could not load model or preprocessor. Make sure files exist. Details: {e}"

    # month validation
    if not 1 <= month <= 12:
        return "Error: Month must be an integer between 1 and 12."

    # generate feature for days in a month
    start_date_str = f"{year}-{month:02d}-01"
    num_days = calendar.monthrange(year, month)[1]
    date_range = pd.to_datetime([f"{year}-{month:02d}-{day:02d}" for day in range(1, num_days + 1)])

    features_list = []
    for date in date_range:
        features_list.append({
            'year': date.year,
            'day_sin': np.sin(2 * np.pi * date.dayofyear / 366),
            'day_cos': np.cos(2 * np.pi * date.dayofyear / 366),
            'month_sin': np.sin(2 * np.pi * date.month / 12),
            'month_cos': np.cos(2 * np.pi * date.month / 12),
            'kecamatan': kecamatan_name
        })
    
    input_df = pd.DataFrame(features_list)

    # preprocess and predict
    try:
        input_processed = preprocessor.transform(input_df)
    except Exception as e:
        return f"Error: Could not process input for '{kecamatan_name}'. Error: {e}"

    all_predicted_values = model.predict(input_processed)

    # calculate average and format output
    average_values = np.mean(all_predicted_values, axis=0)
    
    targets = ['precipprob', 'windspeed', 'temp', 'humidity']
    result = {
        'year': year,
        'month': calendar.month_name[month],
        'kecamatan': kecamatan_name,
    }
    for target, value in zip(targets, average_values):
        result[f"average_{target}"] = round(float(value), 2)
        
    return result

In [6]:
monthly_forecast = predict_monthly_average(
    kecamatan_name="berastagi",
    month=12,
    year=2026
)

monthly_forecast

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step


{'year': 2026,
 'month': 'December',
 'kecamatan': 'berastagi',
 'average_precipprob': 70.26,
 'average_windspeed': 18.57,
 'average_temp': 27.84,
 'average_humidity': 88.85}